In [55]:
import pandas as pd
import getpass
import pymysql 

In [56]:
# Establish the database connection

host = "db.data-science-ust.net"
user = "dimitri"    # !! Replace with your user name !!
password = getpass.getpass()
conn = pymysql.connect(host=host, user=user, passwd=password)
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)

········


In [57]:
cursor.execute('USE shared_sales')

0

In [7]:
cursor.execute("""
SELECT * FROM orders
""")
pd.DataFrame.from_records(cursor.fetchall(), index=["OrderNumber"] )

,OrderDate,ShipDate,CustomerID,EmployeeID
OrderNumber,,,,
1,2017-09-02,2017-09-05,1018,707
2,2017-09-02,2017-09-04,1001,703
3,2017-09-02,2017-09-05,1002,707
4,2017-09-02,2017-09-04,1009,703
5,2017-09-02,2017-09-02,1024,708
...,...,...,...,...
940,2018-03-01,2018-03-05,1007,702
941,2018-03-01,2018-03-01,1019,702
942,2018-03-01,2018-03-02,1002,706


In [20]:
# Customers from California

cursor.execute("""
SELECT CustomerID 
   FROM customers
   WHERE CustState="CA"
""")
pd.DataFrame.from_records(cursor.fetchall(),  index=["CustomerID"] )

""
CustomerID
1008
1013
1014
1015
1018
1019
1022


In [29]:
# Orders for customers from California

cursor.execute("""
SELECT * 
FROM orders
WHERE CustomerID IN (
    SELECT CustomerID 
    FROM customers
    WHERE CustCity="Redmond" AND CustState = "WA")
""")
pd.DataFrame.from_records(cursor.fetchall())

,OrderNumber,OrderDate,ShipDate,CustomerID,EmployeeID
0,2,2017-09-02,2017-09-04,1001,703
1,7,2017-09-02,2017-09-05,1001,708
2,16,2017-09-03,2017-09-07,1001,707
3,52,2017-09-10,2017-09-12,1001,701
4,55,2017-09-10,2017-09-12,1001,707
...,...,...,...,...,...
106,776,2018-02-01,2018-02-01,1006,708
107,824,2018-02-10,2018-02-11,1006,701
108,880,2018-02-19,2018-02-23,1006,704
109,891,2018-02-21,2018-02-22,1006,706


In [30]:
# Customers that have not ordered anything

cursor.execute("""
SELECT * 
FROM customers
WHERE CustomerID NOT IN (
    SELECT CustomerID 
    FROM orders)
""")
pd.DataFrame.from_records(cursor.fetchall())

,CustomerID,CustFirstName,CustLastName,CustStreetAddress,CustCity,CustState,CustZipCode,CustAreaCode,CustPhoneNumber
0,1028,Jeffrey,Tirekicker,19541 104th Ave NE,Bothell,WA,98006,425,555-9999


In [40]:
cursor.execute('SHOW CREATE TABLE orders')
print(cursor.fetchone()['Create Table'])

CREATE TABLE `orders` (
  `OrderNumber` int(11) NOT NULL DEFAULT '0',
  `OrderDate` date DEFAULT NULL,
  `ShipDate` date DEFAULT NULL,
  `CustomerID` int(11) DEFAULT '0',
  `EmployeeID` int(11) DEFAULT '0',
  PRIMARY KEY (`OrderNumber`),
  KEY `CustomerID` (`CustomerID`),
  KEY `EmployeeID` (`EmployeeID`),
  CONSTRAINT `Orders_FK00` FOREIGN KEY (`CustomerID`) REFERENCES `customers` (`CustomerID`),
  CONSTRAINT `Orders_FK01` FOREIGN KEY (`EmployeeID`) REFERENCES `employees` (`EmployeeID`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1


In [41]:
cursor.execute("""
CREATE TABLE regulars(
CustomerID int NOT NULL,
PRIMARY KEY (CustomerID),
FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID)
)
""")

0

In [43]:
# Customers that have not ordered anything

cursor.execute("""
SELECT * 
FROM orders
""")
pd.DataFrame.from_records(cursor.fetchall())

,OrderNumber,OrderDate,ShipDate,CustomerID,EmployeeID
0,1,2017-09-02,2017-09-05,1018,707
1,2,2017-09-02,2017-09-04,1001,703
2,3,2017-09-02,2017-09-05,1002,707
3,4,2017-09-02,2017-09-04,1009,703
4,5,2017-09-02,2017-09-02,1024,708
...,...,...,...,...,...
939,940,2018-03-01,2018-03-05,1007,702
940,941,2018-03-01,2018-03-01,1019,702
941,942,2018-03-01,2018-03-02,1002,706
942,943,2018-03-01,2018-03-02,1026,707


In [52]:
# Give me all the orders that contain an item of over $1000.00 ordered since 2018-01-01
cursor.execute("""
SELECT * 
FROM orders
WHERE OrderDate >= "2018-01-01" AND OrderNumber IN (
    SELECT OrderNumber
    FROM order_details
    WHERE QuotedPrice >= 1000)
""")
pd.DataFrame.from_records(cursor.fetchall())

,OrderNumber,OrderDate,ShipDate,CustomerID,EmployeeID
0,597,2018-01-01,2018-01-01,1006,707
1,598,2018-01-01,2018-01-02,1013,701
2,599,2018-01-02,2018-01-02,1011,703
3,600,2018-01-02,2018-01-06,1006,704
4,602,2018-01-02,2018-01-06,1025,708
...,...,...,...,...,...
217,940,2018-03-01,2018-03-05,1007,702
218,941,2018-03-01,2018-03-01,1019,702
219,942,2018-03-01,2018-03-02,1002,706
220,943,2018-03-01,2018-03-02,1026,707


In [65]:
# Give me orders with customer information 
cursor.execute("""
SELECT * 
FROM orders JOIN customers
ON orders.CustomerID = customers.CustomerID 
""")
pd.DataFrame.from_records(cursor.fetchall())

,OrderNumber,OrderDate,ShipDate,CustomerID,EmployeeID,customers.CustomerID,CustFirstName,CustLastName,CustStreetAddress,CustCity,CustState,CustZipCode,CustAreaCode,CustPhoneNumber
0,2,2017-09-02,2017-09-04,1001,703,1001,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
1,7,2017-09-02,2017-09-05,1001,708,1001,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
2,16,2017-09-03,2017-09-07,1001,707,1001,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
3,52,2017-09-10,2017-09-12,1001,701,1001,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
4,55,2017-09-10,2017-09-12,1001,707,1001,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,829,2018-02-11,2018-02-12,1027,701,1027,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
940,847,2018-02-14,2018-02-16,1027,706,1027,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
941,884,2018-02-20,2018-02-23,1027,704,1027,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
942,886,2018-02-21,2018-02-22,1027,708,1027,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316


In [67]:
# Give me orders with customer information 
cursor.execute("""
SELECT * 
FROM orders JOIN customers
USING (CustomerID)
""")
pd.DataFrame.from_records(cursor.fetchall())

,CustomerID,OrderNumber,OrderDate,ShipDate,EmployeeID,CustFirstName,CustLastName,CustStreetAddress,CustCity,CustState,CustZipCode,CustAreaCode,CustPhoneNumber
0,1001,2,2017-09-02,2017-09-04,703,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
1,1001,7,2017-09-02,2017-09-05,708,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
2,1001,16,2017-09-03,2017-09-07,707,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
3,1001,52,2017-09-10,2017-09-12,701,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
4,1001,55,2017-09-10,2017-09-12,707,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,1027,829,2018-02-11,2018-02-12,701,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
940,1027,847,2018-02-14,2018-02-16,706,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
941,1027,884,2018-02-20,2018-02-23,704,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
942,1027,886,2018-02-21,2018-02-22,708,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316


In [68]:
# Give me orders with customer information 
cursor.execute("""
SELECT * 
FROM orders NATURAL JOIN customers
""")
pd.DataFrame.from_records(cursor.fetchall())

,CustomerID,OrderNumber,OrderDate,ShipDate,EmployeeID,CustFirstName,CustLastName,CustStreetAddress,CustCity,CustState,CustZipCode,CustAreaCode,CustPhoneNumber
0,1001,2,2017-09-02,2017-09-04,703,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
1,1001,7,2017-09-02,2017-09-05,708,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
2,1001,16,2017-09-03,2017-09-07,707,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
3,1001,52,2017-09-10,2017-09-12,701,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
4,1001,55,2017-09-10,2017-09-12,707,Suzanne,Viescas,"15127 NE 24th, #383",Redmond,WA,98052,425,555-2686
...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,1027,829,2018-02-11,2018-02-12,701,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
940,1027,847,2018-02-14,2018-02-16,706,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
941,1027,884,2018-02-20,2018-02-23,704,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316
942,1027,886,2018-02-21,2018-02-22,708,Luke,Patterson,877 145th Ave SE,Portland,OR,97208,503,555-2316


In [72]:
# Give me products with category description included
cursor.execute("""
SELECT ProductNumber, CategoryDescription
FROM categories NATURAL JOIN products""")
cursor.fetchone()

{'ProductNumber': 3, 'CategoryDescription': 'Accessories'}

In [88]:
# Give me bowling teams with their captains' names

cursor.execute("""
SELECT TeamName, BowlerLastName as CaptainLastName, BowlerFirstName as CaptainFirstName
FROM shared_bowling.Teams JOIN shared_bowling.Bowlers
ON CaptainID = BowlerID
""")
pd.DataFrame.from_records(cursor.fetchall())

,TeamName,CaptainLastName,CaptainFirstName
0,Marlins,Fournier,David
1,Sharks,Patterson,Ann
2,Terrapins,Viescas,Carol
3,Barracudas,Sheskey,Richard
4,Dolphins,Viescas,Suzanne
5,Orcas,Thompson,Sarah
6,Manatees,Viescas,Michael
7,Swordfish,Rosales,Joe
8,Huckleberrys,Viescas,David
9,MintJuleps,Hallmark,Alaina


In [77]:
cursor.execute("""
SELECT *
FROM shared_bowling.Bowlers""")
pd.DataFrame.from_records(cursor.fetchall())

,BowlerID,BowlerLastName,BowlerFirstName,BowlerMiddleInit,BowlerAddress,BowlerCity,BowlerState,BowlerZip,BowlerPhoneNumber,TeamID
0,1,Fournier,Barbara,None,67 Willow Drive,Bothell,WA,98123,(206) 555-9876,1
1,2,Fournier,David,None,67 Willow Drive,Bothell,WA,98123,(206) 555-9876,1
2,3,Kennedy,John,A,2957 W 33rd,Ballard,WA,98099,(206) 555-7854,1
3,4,Sheskey,Sara,J,17950 N 59th,Seattle,WA,98011,(206) 555-9893,1
4,5,Patterson,Ann,K,16 Maple Lane,Auburn,WA,98002,(206) 555-3487,2
5,6,Patterson,Neil,None,16 Maple Lane,Auburn,WA,98002,(206) 555-3487,2
6,7,Viescas,David,A,16679 NE 42nd Court,Redmond,WA,98052,(206) 882-8878,2
7,8,Viescas,Stephanie,None,16679 NE 42nd Court,Redmond,WA,98052,(206) 882-8878,2
8,9,Black,Alastair,None,4726 - 11th Ave. N.E.,Seattle,WA,98105,(206) 555-1189,3
9,10,Steele,Doug,None,4110 Old Redmond Rd.,Redmond,WA,98052,(206) 555-8122,3
